# PART_5 (Dataset_coding_id)

## load

In [3]:
%load_ext google.cloud.bigquery

In [4]:
%reload_ext google.cloud.bigquery

In [5]:
import os
import pandas as pd
from google.cloud import bigquery
import google.auth
from google.cloud import bigquery

In [6]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'aqueous-freedom-398208-cfe0d4e567f2.json'

In [7]:
bigquery_client = bigquery.Client()

## revenue harian perkota

In [17]:
query1 = """  
SELECT
    t.transaction_date,
    r.kota AS city,
    SUM(t.price) AS daily_revenue
FROM`aqueous-freedom-398208.ok_food_dataset.transaction_2024` t
INNER JOIN`aqueous-freedom-398208.ok_food_dataset.resto` r ON t.resto_id = r.id

GROUP BY t.transaction_date, r.kota
ORDER BY t.transaction_date, r.kota;

"""
 

result1 = bigquery_client.query(query1)
df1 = result1.to_dataframe()
df1

,transaction_date,city,daily_revenue
0,2024-01-02,Surabaya,223052
1,2024-01-03,Bandung,111359
2,2024-01-03,Surabaya,277131
3,2024-01-04,Bandung,171451
4,2024-01-04,Jakarta,291440
...,...,...,...
116,2024-02-29,Bandung,188567
117,2024-02-29,Jakarta,455069
118,2024-03-01,Denpasar,1836172
119,2024-03-01,Jakarta,137290


## customer segmentation

In [99]:
query3 = """
drop table if exists `aqueous-freedom-398208.OFD_Team_1.coding_id_customer_segmentasi`;
CREATE TABLE `aqueous-freedom-398208.OFD_Team_1.coding_id_customer_segmentasi`as
WITH tabel AS (
    SELECT
        c.id AS customer_id,
        c.name,
        DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY) AS last_order_date,
        COUNT(t.id) AS total_transaction,
        COALESCE(SUM(t.price), 0) AS total_spending
    FROM aqueous-freedom-398208.ok_food_dataset.customer c
    LEFT JOIN aqueous-freedom-398208.ok_food_dataset.transaction_2024 t ON c.id = t.customer_id
    WHERE t.transaction_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY)
    GROUP BY c.id, c.name
)
SELECT
    customer_id,
    name,
    DATE_SUB(CURRENT_DATE(), INTERVAL 0 DAY) AS today_date,
    DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY) AS last_order,
    total_transaction,
    total_spending,
    CASE 
        WHEN total_spending >= 500001 THEN 'Big Spender'
        WHEN total_transaction <= 1 THEN 'New'
        WHEN total_transaction <= 5 THEN 'Good'
        WHEN total_transaction <= 10 THEN 'Potential'
        ELSE 'Loyalist'
    END AS segmentation
FROM tabel
ORDER BY last_order_date DESC;


"""

result3 = bigquery_client.query(query3)
df3 = result3.to_dataframe()
df3

""


## Growth Harian Perkota

In [100]:
query3 = """
drop table if exists `aqueous-freedom-398208.OFD_Team_1.coding_id__growth_harian`;
CREATE TABLE `aqueous-freedom-398208.OFD_Team_1.coding_id__growth_harian`as
WITH DailyRevenue AS (
    SELECT
        r.kota AS Kota,
        t.transaction_date AS Tanggal,
        SUM(t.price) AS Revenue,
        LAG(SUM(t.price), 1) OVER (PARTITION BY r.kota ORDER BY t.transaction_date) AS Revenue_Hari_Kemarin,
        LAG(SUM(t.price), 2) OVER (PARTITION BY r.kota ORDER BY t.transaction_date) AS Revenue_Dua_Hari_Sebelumnya
    FROM
        `aqueous-freedom-398208.ok_food_dataset.transaction_2024` t
    INNER JOIN
        `aqueous-freedom-398208.ok_food_dataset.resto` r ON t.resto_id = r.id
    GROUP BY
        r.kota, t.transaction_date
)

SELECT
    Kota,
    Tanggal,
    COALESCE(Revenue_Hari_Kemarin, Revenue_Dua_Hari_Sebelumnya, 0) AS Revenue_Hari_Kemarin,
    Revenue,
    CASE
        WHEN COALESCE(Revenue_Hari_Kemarin, Revenue_Dua_Hari_Sebelumnya) IS NULL THEN NULL
        ELSE CONCAT(ROUND(((Revenue - COALESCE(Revenue_Hari_Kemarin, Revenue_Dua_Hari_Sebelumnya)) / NULLIF(COALESCE(Revenue_Hari_Kemarin, Revenue_Dua_Hari_Sebelumnya), 0)) * 100, 2), '%')
    END AS Growth_Rate_Percentage
FROM
    DailyRevenue
ORDER BY
    Kota, Tanggal;

"""

result3 = bigquery_client.query(query3)
df3 = result3.to_dataframe()
df3

""


## kontribusi harian per kota

In [101]:
query4 = """
drop table if exists `aqueous-freedom-398208.OFD_Team_1.coding_id_kontribusi_harian_perkota`;
CREATE TABLE `aqueous-freedom-398208.OFD_Team_1.coding_id_kontribusi_harian_perkota`as
WITH city_daily_revenue AS (
    SELECT
        t.transaction_date AS date,
        r.kota AS city,
        SUM(t.price) AS total_revenue
    FROM
        `aqueous-freedom-398208.ok_food_dataset.transaction_2024` t
    INNER JOIN
        `aqueous-freedom-398208.ok_food_dataset.resto` r ON t.resto_id = r.id
    GROUP BY
        t.transaction_date, r.kota
),
daily_total AS (
    SELECT
        transaction_date AS order_date,
        SUM(price) AS total_revenue_all_cities
    FROM
        `aqueous-freedom-398208.ok_food_dataset.transaction_2024`
    GROUP BY
        transaction_date
)
SELECT
    cdr.date,
    cdr.city,
    cdr.total_revenue,
    ROUND((cdr.total_revenue / dt.total_revenue_all_cities) * 100, 0) AS contribution
FROM
    city_daily_revenue AS cdr    
JOIN
    daily_total AS dt ON cdr.date = dt.order_date
ORDER BY
    cdr.date, cdr.city;

"""
result4 = bigquery_client.query(query4)
df4 = result4.to_dataframe()
df4


""
